In [1]:
! pip install pyspark
! pip install findspark

     |████████████████████████████████| 198 kB 3.4 MB/s eta 0:00:01


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("Semi_Sructured_Data_Analysis") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

### Reading CSV


In [3]:
! wget 'https://covid.ourworldindata.org/data/owid-covid-data.csv'

--2021-04-26 10:15:31--  https://covid.ourworldindata.org/data/owid-covid-data.csv
Resolving covid.ourworldindata.org (covid.ourworldindata.org)... 104.21.233.138, 104.21.233.137, 2606:4700:3038::6815:e98a, ...
Connecting to covid.ourworldindata.org (covid.ourworldindata.org)|104.21.233.138|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘owid-covid-data.csv’

owid-covid-data.csv     [           <=>      ]  20.75M  5.05MB/s    in 5.0s    

2021-04-26 10:15:37 (4.12 MB/s) - ‘owid-covid-data.csv’ saved [21756961]



In [5]:
df = spark.read.csv('data/owid-covid-data.csv', sep= ',', header= True)
df.count()

84061

In [12]:
df.select("continent", "location", "date",
          "total_cases", "total_deaths",
          "total_cases_per_million",
          "population"
         ).show(5)

+---------+-----------+----------+-----------+------------+-----------------------+----------+
|continent|   location|      date|total_cases|total_deaths|total_cases_per_million|population|
+---------+-----------+----------+-----------+------------+-----------------------+----------+
|     Asia|Afghanistan|2020-02-24|        1.0|        null|                  0.026|38928341.0|
|     Asia|Afghanistan|2020-02-25|        1.0|        null|                  0.026|38928341.0|
|     Asia|Afghanistan|2020-02-26|        1.0|        null|                  0.026|38928341.0|
|     Asia|Afghanistan|2020-02-27|        1.0|        null|                  0.026|38928341.0|
|     Asia|Afghanistan|2020-02-28|        1.0|        null|                  0.026|38928341.0|
+---------+-----------+----------+-----------+------------+-----------------------+----------+
only showing top 5 rows



In [14]:
df.createOrReplaceTempView("COVID")
covid_data=spark.sql("SELECT location AS country, date, total_cases, \
            total_cases_per_million, population \
            FROM COVID WHERE location='India'")
covid_data.show()

+-------+----------+-----------+-----------------------+------------+
|country|      date|total_cases|total_cases_per_million|  population|
+-------+----------+-----------+-----------------------+------------+
|  India|2020-01-30|        1.0|                  0.001|1380004385.0|
|  India|2020-01-31|        1.0|                  0.001|1380004385.0|
|  India|2020-02-01|        1.0|                  0.001|1380004385.0|
|  India|2020-02-02|        2.0|                  0.001|1380004385.0|
|  India|2020-02-03|        3.0|                  0.002|1380004385.0|
|  India|2020-02-04|        3.0|                  0.002|1380004385.0|
|  India|2020-02-05|        3.0|                  0.002|1380004385.0|
|  India|2020-02-06|        3.0|                  0.002|1380004385.0|
|  India|2020-02-07|        3.0|                  0.002|1380004385.0|
|  India|2020-02-08|        3.0|                  0.002|1380004385.0|
|  India|2020-02-09|        3.0|                  0.002|1380004385.0|
|  India|2020-02-10|

In [19]:
df.createOrReplaceTempView("COVID")
covid_data= spark.sql("SELECT location AS country, date, total_cases,\
            total_cases_per_million, population \
            FROM COVID WHERE location ='India'")
covid_data.show(5)

+-------+----------+-----------+-----------------------+------------+
|country|      date|total_cases|total_cases_per_million|  population|
+-------+----------+-----------+-----------------------+------------+
|  India|2020-01-30|        1.0|                  0.001|1380004385.0|
|  India|2020-01-31|        1.0|                  0.001|1380004385.0|
|  India|2020-02-01|        1.0|                  0.001|1380004385.0|
|  India|2020-02-02|        2.0|                  0.001|1380004385.0|
|  India|2020-02-03|        3.0|                  0.002|1380004385.0|
+-------+----------+-----------+-----------------------+------------+
only showing top 5 rows



In [20]:
covid_data.columns

['country', 'date', 'total_cases', 'total_cases_per_million', 'population']

In [21]:
spark.sql("SHOW COLUMNS FROM COVID")

DataFrame[col_name: string]

In [22]:
spark.sql("DESCRIBE COVID")

DataFrame[col_name: string, data_type: string, comment: string]

In [30]:
# Window Function SQL: Think of a use case
df.createOrReplaceTempView("COVID")
covid_data= spark.sql("SELECT location AS country,date, \
            ROW_NUMBER() OVER (PARTITION BY location ORDER BY date desc) AS row_number \
            FROM COVID WHERE row_number=1")
covid_data.show(100)

AnalysisException: cannot resolve '`row_number`' given input columns: [covid.aged_65_older, covid.aged_70_older, covid.cardiovasc_death_rate, covid.continent, covid.date, covid.diabetes_prevalence, covid.extreme_poverty, covid.female_smokers, covid.gdp_per_capita, covid.handwashing_facilities, covid.hosp_patients, covid.hosp_patients_per_million, covid.hospital_beds_per_thousand, covid.human_development_index, covid.icu_patients, covid.icu_patients_per_million, covid.iso_code, covid.life_expectancy, covid.location, covid.male_smokers, covid.median_age, covid.new_cases, covid.new_cases_per_million, covid.new_cases_smoothed, covid.new_cases_smoothed_per_million, covid.new_deaths, covid.new_deaths_per_million, covid.new_deaths_smoothed, covid.new_deaths_smoothed_per_million, covid.new_tests, covid.new_tests_per_thousand, covid.new_tests_smoothed, covid.new_tests_smoothed_per_thousand, covid.new_vaccinations, covid.new_vaccinations_smoothed, covid.new_vaccinations_smoothed_per_million, cov

### Reading Tab Separated Values (TSV)

In [57]:
! wget 'https://github.com/thepanacealab/covid19_twitter/blob/master/dailies/2021-04-19/2021-04-19_clean-dataset.tsv.gz'
! gzip 'data/2021-04-19_clean-dataset.tsv.gz'

gzip: data/2021-04-19_clean-dataset.tsv.gz already has .gz suffix -- unchanged


In [9]:
# Some other TSV files to play
#! wget 'https://datasets.imdbws.com/title.ratings.tsv.gz' .
#! wget 'https://datasets.imdbws.com/title.ratings.tsv.gz' .
#! gunzip 'data/title.basics.tsv.gz'
#! gunzip 'data/title.ratings.tsv.gz'

In [62]:
df = spark.read.csv("data/2021-04-19-dataset.tsv", header= True, sep= '\t')
df.count()

1645911

In [60]:
df.printSchema()

root
 |-- tweet_id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- time: string (nullable = true)
 |-- lang: string (nullable = true)
 |-- country_code: string (nullable = true)



In [63]:
df.show(4)

+-------------------+----------+--------+----+------------+
|           tweet_id|      date|    time|lang|country_code|
+-------------------+----------+--------+----+------------+
|1383994244709314567|2021-04-19|04:01:55|  en|        NULL|
|1383994244763844612|2021-04-19|04:01:55|  en|        NULL|
|1383994244986130433|2021-04-19|04:01:56|  es|        NULL|
|1383994245065756673|2021-04-19|04:01:56|  in|        NULL|
+-------------------+----------+--------+----+------------+
only showing top 4 rows



### Reading JSON



In [52]:
! wget "https://data.cdc.gov/resource/3apk-4u4f.json"

--2021-04-21 01:54:38--  https://data.cdc.gov/resource/3apk-4u4f.json
Resolving data.cdc.gov (data.cdc.gov)... 52.206.140.199, 52.206.68.26, 52.206.140.205
Connecting to data.cdc.gov (data.cdc.gov)|52.206.140.199|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/json]
Saving to: ‘3apk-4u4f.json’

3apk-4u4f.json          [ <=>                ]  33,63K  --.-KB/s    in 0,002s  

2021-04-21 01:54:40 (15,6 MB/s) - ‘3apk-4u4f.json’ saved [34441]



### Binary Files

In [ ]:
spark.read.format("binaryFile").option("pathGlobFilter", "*.png").load("/data")



##### Number of Delays:
1. **Carrier**: The number of delays and cancellations due to circumstances within the airline's control (e.g. maintenance or crew problems, aircraft cleaning, baggage loading, fueling, etc.) in this month.
2. **Late Aircraft**: The number of delays and cancellations caused by a previous flight with the same aircraft arriving late, causing the present flight to depart late in this month.	
3. **National Aviation System**: The number of delays and cancellations attributable to the national aviation system that refer to a broad set of conditions, such as non-extreme weather conditions, airport operations, heavy traffic volume, and air traffic control in this month.
4. **Security**: Number of delays or cancellations caused by evacuation of a terminal or concourse, re-boarding of aircraft because of security breach, inoperative screening equipment and/or long lines in excess of 29 minutes at screening areas in this month.
5. **Weather**: Number of delays or cancellations caused by significant meteorological conditions (actual or forecasted) that, in the judgment of the carrier, delays or prevents the operation of a flight such as tornado, blizzard or hurricane in this month.	

In [82]:
import requests, json

url = 'https://think.cs.vt.edu/corgis/datasets/json/airlines/airlines.json'
resp = requests.get(url=url)
json_string = json.dumps(resp.json())
json_data= json.loads(json_string)

with open('airlines.json', 'w') as json_file:
    json.dump(json_data, json_file)

In [83]:
df = spark.read.json("airlines.json")
df.show(6)

+--------------------+--------------------+--------------------+
|             Airport|          Statistics|                Time|
+--------------------+--------------------+--------------------+
|{ATL, Atlanta, GA...|{{1009, 1275, 321...|{2003/06, 6, June...|
|{BOS, Boston, MA:...|{{374, 495, 685, ...|{2003/06, 6, June...|
|{BWI, Baltimore, ...|{{296, 477, 389, ...|{2003/06, 6, June...|
|{CLT, Charlotte, ...|{{300, 472, 735, ...|{2003/06, 6, June...|
|{DCA, Washington,...|{{283, 268, 487, ...|{2003/06, 6, June...|
|{DEN, Denver, CO:...|{{516, 323, 664, ...|{2003/06, 6, June...|
+--------------------+--------------------+--------------------+
only showing top 6 rows



In [84]:
df= spark.read.option("multiline","true").json("airlines.json")
#df.show(6)

In [85]:
# Understanding the schema of the JSON through the dataframe
df.printSchema()

root
 |-- Airport: struct (nullable = true)
 |    |-- Code: string (nullable = true)
 |    |-- Name: string (nullable = true)
 |-- Statistics: struct (nullable = true)
 |    |-- # of Delays: struct (nullable = true)
 |    |    |-- Carrier: long (nullable = true)
 |    |    |-- Late Aircraft: long (nullable = true)
 |    |    |-- National Aviation System: long (nullable = true)
 |    |    |-- Security: long (nullable = true)
 |    |    |-- Weather: long (nullable = true)
 |    |-- Carriers: struct (nullable = true)
 |    |    |-- Names: string (nullable = true)
 |    |    |-- Total: long (nullable = true)
 |    |-- Flights: struct (nullable = true)
 |    |    |-- Cancelled: long (nullable = true)
 |    |    |-- Delayed: long (nullable = true)
 |    |    |-- Diverted: long (nullable = true)
 |    |    |-- On Time: long (nullable = true)
 |    |    |-- Total: long (nullable = true)
 |    |-- Minutes Delayed: struct (nullable = true)
 |    |    |-- Carrier: long (nullable = true)
 |    |  

In [86]:

# Subsetting the dataframe to analyse the statistics of # (number) of delays
airport_delay_statistics = df.select("Airport.Code", "Airport.Name", 
                                     "Statistics.# of Delays.Carrier", 
                                     "Statistics.# of Delays.Late Aircraft",
                                     "Statistics.# of Delays.Security", 
                                     "Statistics.# of Delays.Weather")
airport_delay_statistics.printSchema()


root
 |-- Code: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Carrier: long (nullable = true)
 |-- Late Aircraft: long (nullable = true)
 |-- Security: long (nullable = true)
 |-- Weather: long (nullable = true)



In [87]:
# Check which input file has created this dataframe
airport_delay_statistics.inputFiles()

['file:///Users/ankitbit/Documents/Lectures/Real-Time-Data-Analysis/airlines.json']

In [88]:

# Creating summary statistics of the dataframe attributes
airport_delay_statistics.select("Carrier", "Late Aircraft",
                                "Security","Weather").describe().show()


+-------+------------------+----------------+-----------------+-----------------+
|summary|           Carrier|   Late Aircraft|         Security|          Weather|
+-------+------------------+----------------+-----------------+-----------------+
|  count|              4408|            4408|             4408|             4408|
|   mean| 574.6324863883848|789.078947368421|  5.5755444646098|78.21687840290382|
| stddev|329.61647461501815| 561.79842030889|6.007046080059749|75.18172623192343|
|    min|               112|              86|               -1|                1|
|    max|              3087|            4483|               94|              812|
+-------+------------------+----------------+-----------------+-----------------+



In [90]:
# Sorting the airport delay statistics based on the Weather
airport_delay_statistics.select("Code", "Name", "Weather").sort("Weather").show(10)

+----+--------------------+-------+
|Code|                Name|Weather|
+----+--------------------+-------+
| MDW|Chicago, IL: Chic...|      1|
| PDX|Portland, OR: Por...|      2|
| BWI|Baltimore, MD: Ba...|      2|
| FLL|Fort Lauderdale, ...|      2|
| PDX|Portland, OR: Por...|      3|
| IAD|Washington, DC: W...|      3|
| MCO|Orlando, FL: Orla...|      4|
| PDX|Portland, OR: Por...|      4|
| FLL|Fort Lauderdale, ...|      4|
| MDW|Chicago, IL: Chic...|      4|
+----+--------------------+-------+
only showing top 10 rows



In [92]:

# Sorting the airport delay statistics based on the Weather
airport_delay_statistics.select("Code", "Name", "Weather").sort("Weather").show(10, truncate= False)

+----+-------------------------------------------------------------------+-------+
|Code|Name                                                               |Weather|
+----+-------------------------------------------------------------------+-------+
|MDW |Chicago, IL: Chicago Midway International                          |1      |
|PDX |Portland, OR: Portland International                               |2      |
|BWI |Baltimore, MD: Baltimore/Washington International Thurgood Marshall|2      |
|FLL |Fort Lauderdale, FL: Fort Lauderdale-Hollywood International       |2      |
|PDX |Portland, OR: Portland International                               |3      |
|IAD |Washington, DC: Washington Dulles International                    |3      |
|MCO |Orlando, FL: Orlando International                                 |4      |
|PDX |Portland, OR: Portland International                               |4      |
|FLL |Fort Lauderdale, FL: Fort Lauderdale-Hollywood International       |4      |
|MDW

In [93]:
airport_delay_statistics.count()

4408

In [95]:

# Find the list of most affected airports due to delays caused by weather
from pyspark.sql.functions import col, sum, count
airport_delay_statistics.select("Code", "Name", "Weather").groupBy("Code").sum("Weather").show(3)#.sort(col("Weather").desc()).show(5, truncate=False)


+----+------------+
|Code|sum(Weather)|
+----+------------+
| DCA|        8504|
| IAH|       13062|
| LGA|       16350|
+----+------------+
only showing top 3 rows



In [96]:

# Using groupby to find the list of least affected unique airports
from pyspark.sql.functions import col, sum, count
airport_delay_statistics.select("Code", "Name", "Weather") \
    .groupBy("Code", "Name").agg(sum("Weather") \
    .alias("Weather")).sort(col("Weather")).show(10, truncate=False)


+----+------------------------------------------------------------+-------+
|Code|Name                                                        |Weather|
+----+------------------------------------------------------------+-------+
|PDX |Portland, OR: Portland International                        |2791   |
|SAN |San Diego, CA: San Diego International                      |5056   |
|FLL |Fort Lauderdale, FL: Fort Lauderdale-Hollywood International|5084   |
|TPA |Tampa, FL: Tampa International                              |5092   |
|IAD |Washington, DC: Washington Dulles International             |5849   |
|SEA |Seattle, WA: Seattle/Tacoma International                   |5953   |
|MIA |Miami, FL: Miami International                              |7487   |
|CLT |Charlotte, NC: Charlotte Douglas International              |7738   |
|MCO |Orlando, FL: Orlando International                          |8124   |
|DCA |Washington, DC: Ronald Reagan Washington National           |8504   |
+----+------

In [97]:

# Using groupby to find the list of most affected unique airports
from pyspark.sql.functions import col, sum, count
airport_delay_statistics.select("Code", "Name", "Weather") \
    .groupBy("Code", "Name").agg(sum("Weather") \
    .alias("Weather")).sort(col("Weather").desc()).show(10, truncate=False)

+----+------------------------------------------------------+-------+
|Code|Name                                                  |Weather|
+----+------------------------------------------------------+-------+
|ATL |Atlanta, GA: Hartsfield-Jackson Atlanta International |40113  |
|DFW |Dallas/Fort Worth, TX: Dallas/Fort Worth International|30476  |
|ORD |Chicago, IL: Chicago O'Hare International             |24358  |
|LGA |New York, NY: LaGuardia                               |16350  |
|DEN |Denver, CO: Denver International                      |15556  |
|EWR |Newark, NJ: Newark Liberty International              |14668  |
|LAX |Los Angeles, CA: Los Angeles International            |14652  |
|IAH |Houston, TX: George Bush Intercontinental/Houston     |13062  |
|BOS |Boston, MA: Logan International                       |11955  |
|SFO |San Francisco, CA: San Francisco International        |11751  |
+----+------------------------------------------------------+-------+
only showing top 10 

In [98]:

# Find the list of most affected airports due to delays caused by security
airport_delay_statistics.select("Code", "Name", "Security") \
    .groupBy("Code", "Name").agg(sum("Security") \
    .alias("Security")).sort(col("Security").desc()).show(10, truncate=False)


+----+------------------------------------------------------+--------+
|Code|Name                                                  |Security|
+----+------------------------------------------------------+--------+
|LAX |Los Angeles, CA: Los Angeles International            |1738    |
|PHX |Phoenix, AZ: Phoenix Sky Harbor International         |1715    |
|IAH |Houston, TX: George Bush Intercontinental/Houston     |1355    |
|LAS |Las Vegas, NV: McCarran International                 |1288    |
|SLC |Salt Lake City, UT: Salt Lake City International      |1201    |
|SEA |Seattle, WA: Seattle/Tacoma International             |1125    |
|DEN |Denver, CO: Denver International                      |1122    |
|DFW |Dallas/Fort Worth, TX: Dallas/Fort Worth International|1098    |
|ATL |Atlanta, GA: Hartsfield-Jackson Atlanta International |1066    |
|ORD |Chicago, IL: Chicago O'Hare International             |1027    |
+----+------------------------------------------------------+--------+
only s

In [99]:
 airport_delay_statistics.columns

['Code', 'Name', 'Carrier', 'Late Aircraft', 'Security', 'Weather']

In [100]:
airport_delay_statistics.dtypes

[('Code', 'string'),
 ('Name', 'string'),
 ('Carrier', 'bigint'),
 ('Late Aircraft', 'bigint'),
 ('Security', 'bigint'),
 ('Weather', 'bigint')]

In [101]:
airport_delay_statistics.cov("Carrier", "Weather")

18080.795857275385

In [102]:
# Creating a summary statistics of the dataframe
airport_delay_statistics.describe().show()

+-------+----+--------------------+------------------+----------------+-----------------+-----------------+
|summary|Code|                Name|           Carrier|   Late Aircraft|         Security|          Weather|
+-------+----+--------------------+------------------+----------------+-----------------+-----------------+
|  count|4408|                4408|              4408|            4408|             4408|             4408|
|   mean|null|                null| 574.6324863883848|789.078947368421|  5.5755444646098|78.21687840290382|
| stddev|null|                null|329.61647461501815| 561.79842030889|6.007046080059749|75.18172623192343|
|    min| ATL|Atlanta, GA: Hart...|               112|              86|               -1|                1|
|    max| TPA|Washington, DC: W...|              3087|            4483|               94|              812|
+-------+----+--------------------+------------------+----------------+-----------------+-----------------+



In [103]:
# Creating a summary statistics by without columns having NAs 
airport_delay_statistics.describe().dropna().show()

+-------+----+--------------------+-------+-------------+--------+-------+
|summary|Code|                Name|Carrier|Late Aircraft|Security|Weather|
+-------+----+--------------------+-------+-------------+--------+-------+
|  count|4408|                4408|   4408|         4408|    4408|   4408|
|    min| ATL|Atlanta, GA: Hart...|    112|           86|      -1|      1|
|    max| TPA|Washington, DC: W...|   3087|         4483|      94|    812|
+-------+----+--------------------+-------+-------------+--------+-------+



In [104]:
# Creating a summary statistics by without columns having NAs 
airport_delay_statistics.describe().drop("Code", "Name").show()

+-------+------------------+----------------+-----------------+-----------------+
|summary|           Carrier|   Late Aircraft|         Security|          Weather|
+-------+------------------+----------------+-----------------+-----------------+
|  count|              4408|            4408|             4408|             4408|
|   mean| 574.6324863883848|789.078947368421|  5.5755444646098|78.21687840290382|
| stddev|329.61647461501815| 561.79842030889|6.007046080059749|75.18172623192343|
|    min|               112|              86|               -1|                1|
|    max|              3087|            4483|               94|              812|
+-------+------------------+----------------+-----------------+-----------------+



### Filtering rows 

* Discuss more about the various logical operations that can take place apart from OR such as AND, >, < etc
* discuss about various programming methods to implement the same operation
* explain how to extract the first row using FIRST
* explain how to extract the first N rows using HEAD
* 

In [316]:
# Find the list of most affected airports due to delays caused by security
# Filtering rows using a WHERE clause
airport_statistics.select("Code", "Name", "Security") \
    .groupBy("Code", "Name").agg(sum("Security") \
    .alias("Security")).sort(col("Security").desc()) \
    .where((col("Code")== "ATL") | (col("Code")== "TPA")) \
    .show(10, truncate=False)

+----+-----------------------------------------------------+--------+
|Code|Name                                                 |Security|
+----+-----------------------------------------------------+--------+
|ATL |Atlanta, GA: Hartsfield-Jackson Atlanta International|1066    |
|TPA |Tampa, FL: Tampa International                       |500     |
+----+-----------------------------------------------------+--------+



In [317]:
# Find the list of most affected airports due to delays caused by security
# Filtering rows using a FILTER clause
airport_statistics.select("Code", "Name", "Security") \
    .groupBy("Code", "Name").agg(sum("Security") \
    .alias("Security")).sort(col("Security").desc()) \
    .filter((col("Code")== "ATL") | (col("Code")== "TPA")) \
    .show(10, truncate=False)

+----+-----------------------------------------------------+--------+
|Code|Name                                                 |Security|
+----+-----------------------------------------------------+--------+
|ATL |Atlanta, GA: Hartsfield-Jackson Atlanta International|1066    |
|TPA |Tampa, FL: Tampa International                       |500     |
+----+-----------------------------------------------------+--------+



In [318]:
# Find the list of most affected airports due to delays caused by security
# Filtering rows using a WHERE clause

condition_1= (col("Code")== "ATL")
condition_2= (col("Code")== "TPA")

airport_statistics.select("Code", "Name", "Security") \
    .groupBy("Code", "Name").agg(sum("Security") \
    .alias("Security")).sort(col("Security").desc()) \
    .where(condition_1 | condition_2) \
    .show(10, truncate=False)

+----+-----------------------------------------------------+--------+
|Code|Name                                                 |Security|
+----+-----------------------------------------------------+--------+
|ATL |Atlanta, GA: Hartsfield-Jackson Atlanta International|1066    |
|TPA |Tampa, FL: Tampa International                       |500     |
+----+-----------------------------------------------------+--------+



In [327]:
airport_delay_statistics.isStreaming

False

In [330]:
airport_delay_statistics.na

In [331]:
airport_delay_statistics.persist

<bound method DataFrame.persist of DataFrame[Code: string, Name: string, Carrier: bigint, Late Aircraft: bigint, Security: bigint, Weather: bigint]>

In [338]:
split_1, split_2= airport_delay_statistics.randomSplit([1.0, 2.0], 24)

In [343]:
split_1.show(3)

+----+--------------------+-------+-------------+--------+-------+
|Code|                Name|Carrier|Late Aircraft|Security|Weather|
+----+--------------------+-------+-------------+--------+-------+
| ATL|Atlanta, GA: Hart...|    776|         1910|       4|    101|
| ATL|Atlanta, GA: Hart...|    875|         1146|       6|     39|
| ATL|Atlanta, GA: Hart...|    880|         1095|       3|     26|
+----+--------------------+-------+-------------+--------+-------+
only showing top 3 rows



In [341]:
split_2.show(3)

+----+--------------------+-------+-------------+--------+-------+
|Code|                Name|Carrier|Late Aircraft|Security|Weather|
+----+--------------------+-------+-------------+--------+-------+
| ATL|Atlanta, GA: Hart...|    700|          807|       4|    161|
| ATL|Atlanta, GA: Hart...|    756|          906|       4|    170|
| ATL|Atlanta, GA: Hart...|    789|          888|       6|    155|
+----+--------------------+-------+-------------+--------+-------+
only showing top 3 rows



In [ ]:
# df.replace
# df.repartition, df.repartitionByRange
# df.registerTempTable
# df.rdd
# df.intersect
# df.intersectAll
# df.collect vs df.take vs df.show
# df.hint()
# using explode feature
# createGlobalTempView
# createOrReplaceGlobalTempView
# distinct 
# drop_duplicates
# exceptALL
# fillna
# 
# cube function is most important